In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# Basics
import tensorflow as tf
import pandas as pd
import xarray as xr

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.DRN.DRN_load_score import *  # Load DRN_scores
from src.models.DRN.DRN_load_model import *  # Load Models
from src.models.DRN.DRN_train import *
from src.models.EMOS_local.EMOS_local_load_score import *  # Load EMOS_local_scores
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models#
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores

2023-06-10 16:51:16.739238: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-10 16:51:16.739261: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### 1. Load train-test_data

In [ ]:
# load train and test data
train_var_denormed = ldpd.load_data_all_train_proc_denorm()
dat_X_train_lead_all_denorm, dat_y_train_lead_all_denorm = split_var_lead(
    train_var_denormed
)
drn_X_train_lead_array, drn_embedding_train_lead_array = make_X_array(
    dat_X_train_lead_all_denorm, 0
)
# Reshape target values into a 1D array
t2m_y_train = dat_y_train_lead_all_denorm[5][0].values.flatten()

# load test data
test_var_denormed = ldpd.load_data_all_test_proc_denorm()
dat_X_test_lead_all_denorm, dat_y_test_lead_all_denorm = split_var_lead(
    test_var_denormed
)
drn_X_test_lead_array, drn_embedding_test_lead_array = make_X_array(
    dat_X_test_lead_all_denorm, 0
)
# Reshape target values into a 1D array
t2m_y_test = dat_y_test_lead_all_denorm[5][0].values.flatten()


In [ ]:
# Load trained models
DRN_ws10_models = DRN_load_model("ws10")

### 2. Try train model:

In [ ]:
DRN_0_model_test = DRN_train(
    5,
    0,
    hidden_layer=[1024],
    emb_size=10,
    batch_size=2048,
    epochs=1,
    lr=0.001,
    save=False,
)

In [ ]:
# Evaluate
DRN_0_test_score = DRN_0_model_test.evaluate(
    [drn_X_test_lead_array, drn_embedding_test_lead_array], t2m_y_test
)